In [ ]:
"""Alunas: Clara Azevedo-GEC-1897; Maria Eduarda de Oliveira-GEC-1896; Vitória Dutra-GES-414"""

import threading
import random

class Player(threading.Thread):
    def __init__(self, player_id, name):
        """Inicializa um jogador com um ID e um nome."""
        super().__init__()
        self.player_id = player_id
        self.name = name
        self.choice = None
        self.opponent = None

    def make_choice(self):
        """Faz uma escolha aleatória entre 'pedra', 'papel' e 'tesoura'."""
        self.choice = random.choice(['pedra', 'papel', 'tesoura'])
        print(f'{self.name} escolheu {self.choice}')

    def run(self):
        """Método que será executado quando a thread for iniciada."""
        if self.opponent:
            self.make_choice()
            self.opponent.make_choice()

    def play_even_odd(self):
        """Joga par ou ímpar com outro jogador."""
        choice = random.choice(['par', 'ímpar'])
        number = random.randint(1, 10)
        print(f'{self.name} escolheu {choice} e jogou o número {number}')
        return choice, number

In [8]:
import threading
import random
import time

# Classe que representa um jogador no jogo, usando threads
class Player(threading.Thread):
    def __init__(self, name, age, opponent, winners_unsafe, winners_safe, lock):
        super().__init__()
        self.name = name
        self.age = age
        self.opponent = opponent
        self.choice = None
        self.winners_unsafe = winners_unsafe
        self.winners_safe = winners_safe
        self.lock = lock

    # Método para o jogador fazer sua escolha
    def make_choice(self):
        time.sleep(random.uniform(0.1, 0.4))  # Simula o tempo de escolha
        self.choice = random.choice(['pedra', 'papel', 'tesoura'])
        print(f'{self.name} escolheu {self.choice}')

    # Método para o jogador jogar par ou ímpar
    def play_even_odd(self):
        time.sleep(random.uniform(0.1, 0.4))  # Simula o tempo de escolha
        choice = random.choice(['par', 'ímpar'])
        number = random.randint(1, 10)
        print(f'{self.name} escolheu {choice} e jogou {number}')
        return choice, number

    # Método para tentar registrar o vencedor nas listas (segurança com e sem lock)
    def try_register_winner(self, winner_name):
        print(f'{self.name} tentando registrar (sem controle)...')
        self.winners_unsafe.append(winner_name)
        print(f'{self.name} registrou {winner_name} (sem controle).')

        print(f'{self.name} tentando registrar (com controle)...')
        with self.lock:
            if winner_name not in self.winners_safe:
                self.winners_safe.append(winner_name)
        print(f'{self.name} registrou {winner_name} (com controle).')

    # Função que define o comportamento do jogador durante o jogo
    def run(self):
        self.make_choice()  # O jogador faz sua escolha
        self.opponent.make_choice()  # Oponente também faz a escolha

        winner = None
        if self.choice == self.opponent.choice:  # Empate, vai para par ou ímpar
            print(f'{self.name} empatou com {self.opponent.name}, vão disputar par ou ímpar.')
            choice1, number1 = self.play_even_odd()
            choice2, number2 = self.opponent.play_even_odd()
            total = number1 + number2
            result = 'par' if total % 2 == 0 else 'ímpar'
            if choice1 == result:
                winner = self.name
            else:
                winner = self.opponent.name
        else:
            # Verifica o vencedor do jogo pedra, papel e tesoura
            if (self.choice == 'pedra' and self.opponent.choice == 'tesoura') or \
               (self.choice == 'papel' and self.opponent.choice == 'pedra') or \
               (self.choice == 'tesoura' and self.opponent.choice == 'papel'):
                winner = self.name
            else:
                winner = self.opponent.name

        # Tenta registrar o vencedor nas listas
        self.try_register_winner(winner)

# Função para simular o jogo com diferentes tipos de escalonamento
def simulate_game(player_data, scheduling='FCFS'):
    if scheduling == 'PS':
        player_data.sort(key=lambda x: -x[1])  # Mais velho primeiro
        print("\n--- Escalonamento PS (por idade) ---")
    else:
        random.shuffle(player_data)  # Embaralha para FCFS
        print("\n--- Escalonamento FCFS ---")

    print("Jogadores:", [p[0] for p in player_data])

    player_names = [p[0] for p in player_data]
    player_ages = {p[0]: p[1] for p in player_data}

    round_num = 1
    start_time = time.time()  # Marca o tempo de início
    while len(player_names) > 1:
        winners_unsafe = []
        winners_safe = []
        lock = threading.Lock()
        threads = []
        player_pairs = []

        # Cria os pares de jogadores e inicializa as threads
        for i in range(0, len(player_names) - 1, 2):
            name1 = player_names[i]
            name2 = player_names[i+1]

            player1 = Player(name1, player_ages[name1], None, winners_unsafe, winners_safe, lock)
            player2 = Player(name2, player_ages[name2], player1, winners_unsafe, winners_safe, lock)
            player1.opponent = player2

            threads.append(player1)
            threads.append(player2)
            player_pairs.append((name1, name2))

        # Inicia as threads
        for t in threads:
            t.start()
        for t in threads:
            t.join()  # Aguarda todas as threads terminarem

        # Atualiza a lista de jogadores com base nos vencedores
        new_player_names = []
        for name in winners_safe:
            if name not in new_player_names:
                new_player_names.append(name)

        player_names = new_player_names  # Atualiza a lista de jogadores para a próxima rodada
        round_num += 1

    elapsed_time = time.time() - start_time  # Calcula o tempo total
    print(f"\nTempo {scheduling}: {elapsed_time:.2f} segundos")
    print(f"\nVencedor: {player_names[0]}\n")
    print(f"Winners (sem controle - {scheduling}):", winners_unsafe)
    print(f"Winners (com controle - {scheduling}):", winners_safe)

# Função principal que inicializa os jogadores e chama a simulação
def main():
    players_with_ages = [
        ('Alice', 20), ('Joao', 25), ('Carla', 22), ('Davi', 28),
        ('Eva', 26), ('Lucas', 21), ('Julia', 23), ('Marcos', 24)
    ]

    # Simula o jogo com os dois tipos de escalonamento
    simulate_game(players_with_ages, scheduling='FCFS')
    simulate_game(players_with_ages, scheduling='PS')

if __name__ == '__main__':
    main()



--- Escalonamento FCFS ---
Jogadores: ['Joao', 'Alice', 'Julia', 'Carla', 'Davi', 'Lucas', 'Eva', 'Marcos']
Alice escolheu pedra
Lucas escolheu tesoura
Carla escolheu tesoura
Davi escolheu papel
Julia escolheu tesoura
Marcos escolheu tesoura
Joao escolheu tesoura
Lucas escolheu pedra
Davi tentando registrar (sem controle)...
Davi registrou Davi (sem controle).
Davi tentando registrar (com controle)...
Davi registrou Davi (com controle).
Julia escolheu tesoura
Carla empatou com Julia, vão disputar par ou ímpar.
Davi escolheu pedra
Lucas empatou com Davi, vão disputar par ou ímpar.
Carla escolheu tesoura
Julia empatou com Carla, vão disputar par ou ímpar.
Eva escolheu tesoura
Marcos empatou com Eva, vão disputar par ou ímpar.
Eva escolheu tesoura
Joao escolheu papel
Alice tentando registrar (sem controle)...
Alice registrou Joao (sem controle).
Alice tentando registrar (com controle)...
Alice registrou Joao (com controle).
Julia escolheu ímpar e jogou 2
Marcos escolheu ímpar e jogou 2
M